# Watchdogs
The Watchdog class is a modular monitoring unit which periodically runs one experiment and compares the result against a defined threshold. Thresholds are defined with a tuple of the lower and upper bound; passing None to either element will deactivate that bound.

In [ ]:
from emergent.monitor import Watchdog

def experiment():
    return 2

params = {}
w = Watchdog(experiment=experiment, threshold=(0, 1), name='MyWatchdog')
w.check()

The check() method returns a tuple consisting of the measured value and the result of the logical comparison to the specified thresholds.

When a check fails, the Watchdog calls its react() method. By default, this method simply returns a null result, but you can implement any custom functionality to respond to problems with monitored variables. For example, let's reimplement w.react() to print a big warning message when the check fails:

In [ ]:
def warning():
    print('WARNING: Watchdog variable out of range!!')
    
w.react = warning
w.check()

Practical implementation of a Watchdog-based monitoring system in the lab will probably involve passing ADC read methods to the Watchdog.experiment attribute. For example, suppose the function read_ADC(ch) reads a channel labeled by the integer 'ch' (we'll return dummy values of 0 and 2 instead):

In [ ]:
def read_ADC(ch):
    return [0.5, 2][ch]

To monitor both channels, we would simply create two Watchdog objects, each pointing at a given channel:

In [ ]:
from functools import partial, partialmethod
watchdogs = {}
for ch in range(2):
    func = partial(read_ADC, ch)
    watchdogs[ch] = Watchdog(func, threshold=(0,1))

print(watchdogs[0].check())
print(watchdogs[1].check())

# Monitors
The Monitor class manages multiple Watchdogs. We'll use it to handle the two objects defined in the previous example. If a filename is passed to the Monitor constructor, the result will automatically be appended to the file in a new line.

In [ ]:
from emergent.monitor import Monitor
        
        
m = Monitor(watchdogs, filename = 'watchdog_test.txt')
m.check()




For continuous periodic monitoring, call the Monitor.start_periodic() method with your desired period. For example, let's check once per second:

In [ ]:
m.start_periodic(1)

Note that the Monitor.start_periodic method uses Python's sched module for event scheduling to avoid accumulated drifts. For example, if your check() method takes 50 ms to return and you call it in a loop with a 1s delay, the true time between checks will actually be 1.05 s. Using the scheduler allows drift-free checking; however, you should make sure that the check() call always returns in a time shorter than your chosen period!

To stop the monitoring, just call the Monitor.stop() method:

In [ ]:
m.stop()

Monitoring can be synced to some other process through software triggers, e.g. TTL pulses acquired with a DAQ board. Just define a method which returns as soon as a trigger is received and pass it into the Monitor.start_triggered() method. Let's simulate a trigger arriving once per second:

In [ ]:
import time
def trigger():
    time.sleep(1)
    return

m.start_triggered(trigger)

In [ ]:
m.stop()

# Core integration

The examples above show how to use the monitor module as a standalone system. Now we will see how to use it in a more powerful context - to watch important signals in your experiment and trigger optimizations if they exit a defined range. First, let's boot up the Core:

In [1]:
%run "./core.ipynb"
import requests
base_url = 'http://localhost:6000'
while True:
    try:
        r=requests.get(base_url)
        if r.text == 'EMERGENT API':
            break
    except:
        continue
print(r.text)

/Users/robbiefasano/emergent/emergent
Overwriting /Users/robbiefasano/emergent/emergent/networks/test/devices/test_device.py
Overwriting /Users/robbiefasano/emergent/emergent/networks/test/hubs/test_hub.py
Overwriting /Users/robbiefasano/emergent/emergent/networks/test/network.py
API running at 127.0.0.1:6000
DataDict([('device', {'X': 1, 'Y': 2})])
New state: DataDict([('device', {'X': 1, 'Y': 2})])
 * Serving Flask app "emergent.API.API" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
EMERGENT API


Now let's define a watchdog and a monitor to check the result of the "gaussian" experiment attached to the hub. We'll also set up a very simple optimization pipeline to handle this (more sophisticated examples are shown in the Optimize tutorial):

In [6]:
from emergent.monitor import Watchdog, Monitor
hub = core.hubs['hub']
experiment = lambda: -hub.gaussian()
w = Watchdog(experiment=experiment, threshold=(0.5, 1), name='gaussian')

from emergent.pipeline import *
import logging as log
def reoptimize(monitor, watchdog, experiment, hub):
    log.warning('Signal below threshold - running optimization')
    monitor.stop()     # pause monitoring while reoptimizing
    state = {'device': hub.state['device']}
    bounds = {'device': {'X': (0,1), 'Y': (0,1)}}
    pipe = Pipeline(state, bounds, experiment)
    pipe.add(LBFGSB())
    points, costs = pipe.run()
    monitor.start_periodic(1)     # restart monitoring

m = Monitor({'gaussian': w}, filename = 'watchdog_test.txt')
w.react = lambda: reoptimize(m, w, hub.gaussian, hub)

hub.actuate({'device': {'X':0.3, 'Y':0.6}})     # set the initial state to the global optimum

m.start_periodic(1)

If you're not familiar with Python's lambda, it allows you define functions in single lines, but defer the execution until the lambda is called - here, we define "experiment" as a method which executes the gaussian experiment and returns the opposite sign.

Since we set the initial state to the point which optimizes the gaussian cost function, the monitor is perfectly happy. But watch what happens if we move it away (which would also happen if some experimental drift compromised the signal):

In [7]:
hub.actuate({'device': {'X':0.0, 'Y':0.6}})     # set the initial state to the global optimum


INFO:root:Optimization complete!
INFO:root:Time: 0s
INFO:root:Evaluations: 19
INFO:root:Initial cost: -0.367879
INFO:root:Final cost: -1.000000
INFO:root:Improvement: 171.8%


In [8]:
hub.state

DataDict([('device', {'X': 0.3000249772106291, 'Y': 0.600000008163276})])

After we degraded the signal by moving the X degree of freedom away from the optimal point, the monitor automatically reoptimized and returned to near the best point! So now you have a system which actively tries to return to its state of best performance - even if you intentionally turn a knob away from this point!